In [1]:
from __future__ import unicode_literals
import youtube_dl
import urllib.request
from bs4 import BeautifulSoup
import requests
from youtube_search import YoutubeSearch
from threading import Thread
import multiprocessing
import json

In [2]:
import sys
import trace
import threading

class KThread(threading.Thread):
  def __init__(self, *args, **keywords):
    threading.Thread.__init__(self, *args, **keywords)
    self.killed = False

  def start(self):
    self.__run_backup = self.run
    self.run = self.__run      
    threading.Thread.start(self)

  def __run(self):
    sys.settrace(self.globaltrace)
    self.__run_backup()
    self.run = self.__run_backup

  def globaltrace(self, frame, why, arg):
    if why == 'call':
      return self.localtrace
    else:
      return None

  def localtrace(self, frame, why, arg):
    if self.killed:
      if why == 'line':
        raise SystemExit()
    return self.localtrace

  def kill(self):
    self.killed = True

In [13]:
import os
from pydub import AudioSegment
from pydub.playback import play

def globalPlay(storage, filename):
    from pydub import AudioSegment
    from pydub.playback import play
    song = AudioSegment.from_mp3(storage + filename)
    play(song)

class MusicPlayer:
    
    def __init__(self):
        self.current_song = None
        self.baseurl = "http://www.youtube.com"
        self.storage = './temp'
        self.filename = '/xxx.mp3'
        self.music_available = True
        self.thread = None
        self.playing = False
        if not os.path.exists(self.storage):
            os.mkdir(self.storage)
        
    def searchQuery(self, query):
        results = YoutubeSearch(query, max_results=1).to_json()
        results = json.loads(results)
        return self.baseurl + results['videos'][0]['url_suffix']
    
    def fetchAudio(self, link):
        ydl_opts = {'format': 'bestaudio/best',
                    'outtmpl': self.storage + self.filename,
                    'postprocessors': [{'key': 'FFmpegExtractAudio','preferredcodec': 'mp3','preferredquality': '192'}]}
        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            ydl.download([link])
        self.music_available = True
            
    def playMusic(self, song_name):
        link = self.searchQuery(song_name)
        self.fetchAudio(link)
        if self.thread is None:
            if not self.playing:
                self.thread = KThread(target=globalPlay, args=[self.storage, self.filename])
                self.thread.start()
                self.playing = True
        
    def stopMusic(self):
        if self.playing and self.thread:
            self.thread.kill()
            self.playing = False
            self.thread = None

In [14]:
player = MusicPlayer()
player.playMusic('sidhu moosewala bad')

[youtube] uzO1ZS9qycA: Downloading webpage
[download] Destination: temp\xxx.mp3
[download] 100% of 3.22MiB in 00:03                  
[ffmpeg] Post-process file temp\xxx.mp3 exists, skipping


Exception in thread Thread-44:
Traceback (most recent call last):
  File "c:\users\asus\appdata\local\programs\python\python37\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "<ipython-input-2-ca305922d80c>", line 17, in __run
    self.__run_backup()
  File "c:\users\asus\appdata\local\programs\python\python37\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-13-385ff9489ceb>", line 8, in globalPlay
    song = AudioSegment.from_mp3(storage + filename)
  File "c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\pydub\audio_segment.py", line 716, in from_mp3
    return cls.from_file(file, 'mp3', parameters=parameters)
  File "c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\pydub\audio_segment.py", line 704, in from_file
    p.returncode, p_err))
pydub.exceptions.CouldntDecodeError: Decoding failed. ffmpeg returned error code: 1

Output from ffmpeg/avlib:

b"ffmpeg versi

In [55]:
player.stopMusic()

In [12]:
song = AudioSegment.from_mp3('./temp/xxx.mp3')
play(song)

CouldntDecodeError: Decoding failed. ffmpeg returned error code: 1

Output from ffmpeg/avlib:

b"ffmpeg version N-94421-gb3b7523feb Copyright (c) 2000-2019 the FFmpeg developers\r\n  built with gcc 9.1.1 (GCC) 20190716\r\n  configuration: --enable-gpl --enable-version3 --enable-sdl2 --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libdav1d --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libmfx --enable-amf --enable-ffnvcodec --enable-cuvid --enable-d3d11va --enable-nv